<a href="https://colab.research.google.com/github/HeNeos/Reports-FIM-UNI/blob/master/MC516-C%C3%A1lculo%20por%20elementos%20finitos/PC3/Truss_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
def NaiveMultiply(A,B):
    C = np.zeros((A.shape[0],B.shape[1]))
    for i in range(0,C.shape[0]):
        for j in range(0,C.shape[1]):
            aux = 0
            for k in range(0,A.shape[1]):
                aux += A[i][k]*B[k][j]
            C[i][j] = aux
    return C
 
def nextPowerofTwo(n):
    return int(2**(ceil(log2(n))))
 
def ModifyMatrix(A):
    newA = A
    if(A.shape[1]%2 == 1):
        aux = np.zeros((A.shape[0],A.shape[1]+1))
        aux = np.insert(A,A.shape[1],0,axis=1)
        newA = aux
    if(newA.shape[0]%2 == 1):
        aux = np.zeros((newA.shape[0]+1,newA.shape[1]))
        aux = np.insert(newA,newA.shape[0],0,axis=0)
        newA = aux
    return newA
 
def FastMultiply(oldA,oldB):
    rows = oldA.shape[0]
    columns = oldB.shape[1]
    if(rows <= 2 or columns <=2 or oldA.shape[1] <= 2 or oldB.shape[0] <= 2):
        return np.matmul(oldA,oldB)
    
    A = ModifyMatrix(oldA)
    B = ModifyMatrix(oldB)
    N1 = A.shape[0]
    N2 = A.shape[1]
    N3 = B.shape[0]
    N4 = B.shape[1]
    
 
    a = A[0:N1//2,0:N2//2]
    b = A[0:N1//2,N2//2:N2//2+N2//2]
    c = A[N1//2:N1//2+N1//2,0:N2//2]
    d = A[N1//2:N1//2+N1//2,N2//2:N2//2+N2//2]
    
    e = B[0:N3//2,0:N4//2]
    f = B[0:N3//2,N4//2:N4//2+N4//2]
    g = B[N3//2:N3//2+N3//2,0:N4//2]
    h = B[N3//2:N3//2+N3//2,N4//2:N4//2+N4//2]
    
    
    p1 = FastMultiply(a,(f-h))
    p3 = FastMultiply((c+d),e)
    p2 = FastMultiply((a+b),h)
    p4 = FastMultiply(d,(g-e))
    p5 = FastMultiply((a+d),(e+h))
    p6 = FastMultiply((b-d),(g+h))
    p7 = FastMultiply((a-c),(e+f))
    
    
    C = np.zeros((rows,columns))
    
    c11 = p5 + p4 - p2 + p6
    c12 = p1 + p2
    c21 = p3 + p4
    c22 = p1 + p5 - p3 - p7
    
 
    
    for i in range(0,N1//2):
        for j in range(0,N4//2):
            C[i][j] = c11[i][j]
            if(j + N4//2 < columns):
                C[i][j+N4//2] = c12[i][j]
            if(i + N1//2 < rows):
                C[i+N1//2][j] = c21[i][j]
                if(j + N4//2 < columns):
                    C[i+N1//2][j+N4//2] = c22[i][j]
    return C

In [ ]:
def conjugate_grad(A, b, x=None):
    n = b.shape[0]
    if not x:
        x = np.ones((n,1))
    r = np.dot(A, x) - b
    p = - r
    r_k_norm = FastMultiply(np.transpose(r), r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        
        alpha = r_k_norm /FastMultiply(np.transpose(p), Ap)
        x += alpha * p
        r += alpha * Ap
        r_kplus1_norm = FastMultiply(np.transpose(r), r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        p = beta * p - r
    return x

In [ ]:
NodesCondition = []
ForcesCondition = []

def DistNodes(f,s):
    if(s[0] == f[0]):
        aux = np.pi/2
        if(s[1] < f[1]):
             aux *= -1
        return (np.sqrt((s[0]-f[0])**2+(s[1]-f[1])**2),aux)
    else:
        aux = np.arctan((s[1]-f[1])/(s[0]-f[0]))
    if(aux < 0 and s[1] > f[1]):
        aux += np.pi
    if(s[1] < f[1]):
        aux += np.pi
        if(s[0] > f[0]):
            aux += np.pi

    return (np.sqrt((s[0]-f[0])**2+(s[1]-f[1])**2),aux)
 
 
def UBoundaryCondition(nU,u,i):
    nU[i][0] = u
    NodesCondition.append(i)
 
def FBoundaryCondition(nF,f,i):
    nF[i][0] += f
    ForcesCondition.append(i)
    
def AssemblyStiffness(nStiffnessMatrix,k,i,j):
    for p in range(0,2):
        for m in range(0,2):
            nStiffnessMatrix[2*i+p][2*i+m] += k[p][m]
            nStiffnessMatrix[2*i+p][2*j+m] += k[p][2+m]
            nStiffnessMatrix[2*j+p][2*i+m] += k[p+2][m]
            nStiffnessMatrix[2*j+p][2*j+m] += k[p+2][2+m]
 
def Initialize(nStiffnessMatrix,nU,nF):
    for i in range(0,Nodes):
        nU[i][0] = 0
        nF[i][0] = 0
    for i in range(0,NumberOfElement):
        AssemblyStiffness(nStiffnessMatrix,K[i],int(Elements[i][0]),int(Elements[i][1]))

def TMatrix(nT,i,angle):
    nT[2*i][2*i] = np.cos(angle)
    nT[2*i][2*i+1] = np.sin(angle)
    nT[2*i+1][2*i] = -np.sin(angle)
    nT[2*i+1][2*i+1] = np.cos(angle)

def ApplyT(nStiffnessMatrix, nT):
    return FastMultiply(FastMultiply(nT,nStiffnessMatrix),np.transpose(nT))

def PreSolvingStiffness(nStiffnessMatrix):
    nsize = Nodes-len(NodesCondition)
    newStiffness = np.zeros((nsize,nsize))
    contr = -1
    for i in range(0,Nodes):
        contc = -1
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        for j in range(0,Nodes):
            flagc = False
            for k in range(0,len(NodesCondition)):
                if(j == NodesCondition[k]):
                    flagc = True
                    break
            if(flagc):
                continue
            contc += 1
            newStiffness[contr][contc] = nStiffnessMatrix[i][j]
    return newStiffness
 
 
def PreSolvingF(nF,nS,nU):
    nsize = Nodes-len(NodesCondition)
    newF = np.zeros(nsize).reshape(nsize,1)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            for k in range(0,Nodes):
                nF[k][0] = nF[k][0]-nS[k][i]*nU[i][0]
            continue
 
            
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        newF[contr][0] = nF[i][0]
    
    return newF
                      
 
def Solve(nStiffnessMatrix,nU,nF):
    newStiffness = PreSolvingStiffness(nStiffnessMatrix)
    newF = PreSolvingF(nF,nStiffnessMatrix,nU)
    u = conjugate_grad(newStiffness,newF)    
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        nU[i][0] = u[contr][0]
    nnF = FastMultiply(StiffnessMatrix,nU)
    return nU,nnF

In [ ]:
NodesCondition = []
Nodes = 5
Nodes *= 2
NumberOfElement = 6

h = 1500 #mm
E = 3.2e5 #MPA
K = []
A = (0.25*np.pi*(50)**2)*np.ones(Nodes) #mm^2
L = []
P_A = 5000 #N
P_B = 4200 #N 
P_C = 2500 #N 
P_E = 3000 #N

PosNodes = np.array([(0,0),(h,0),(0,h),(h,h),(h,2*h)])
Elements = np.array([(0,2),(1,2),(1,3),(2,3),(2,4),(3,4)])

for i in range(0,NumberOfElement):
    L.append(DistNodes(PosNodes[Elements[i][0]],PosNodes[Elements[i][1]]))

L = np.array(L)


for i in range(0,NumberOfElement):
    aux = np.zeros((4,4))
    angle = L[i][1]
    rows = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    cols = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    for j in range(0,4):
        for k in range(0,4):
            aux[j][k] = rows[j]*cols[k]
    aux = aux*E*A[i]/L[i][0]
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(x=0,y=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en X
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+0) #Nodo 3 en X
UBoundaryCondition(U,0,2*1+1) #Nodo 3 en Y

FBoundaryCondition(F,-P_C,2*2+0) #Nodo 2 en X
FBoundaryCondition(F,-P_E,2*3+0) #Nodo 3 en X
FBoundaryCondition(F,-P_B,2*4+0) #Nodo 4 en X
FBoundaryCondition(F,P_A,2*4+1) #Nodo 4 en Y

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)


Stiffness Matrix:
 [[ 1.57054477e-27  2.56489426e-11  0.00000000e+00  0.00000000e+00
  -1.57054477e-27 -2.56489426e-11  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.56489426e-11  4.18879020e+05  0.00000000e+00  0.00000000e+00
  -2.56489426e-11 -4.18879020e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.48096098e+05 -1.48096098e+05
  -1.48096098e+05  1.48096098e+05 -1.57054477e-27 -2.56489426e-11
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.48096098e+05  5.66975118e+05
   1.48096098e+05 -1.48096098e+05 -2.56489426e-11 -4.18879020e+05
   0.00000000e+00  0.00000000e+00]
 [-1.57054477e-27 -2.56489426e-11 -1.48096098e+05  1.48096098e+05
   7.15071216e+05  2.91038305e-11 -4.18879020e+05  0.00000000e+00
  -1.48096098e+05 -1.48096098e+05]
 [-2.56489426e-11 -4.18879020e+05  1.48096098e+05 -1.48096098e+05
   2.91038305e-11  7.15071216e+05  0.00000000e+00  0.00000000e+00
  -1.48096098e

In [ ]:
ElementDisplacement = []
for i in range(0,NumberOfElement):
    Node1 = Elements[i][0]
    Node2 = Elements[i][1]
    ElementDisplacement.append([(U[2*Node1][0],U[2*Node1+1][0]),(U[2*Node2][0],U[2*Node2+1][0])])
Stress = []
for i in range(0,NumberOfElement):
    C = np.cos(L[i][1])
    S = np.sin(L[i][1])
    dd = ElementDisplacement[i]
    print(i,L[i][1])
    ss = -dd[0][0]*C - dd[0][1]*S + dd[1][0]*C+ dd[1][1]*S
    Stress.append(ss*E/L[i][0])
Stress = np.array(Stress)
print("Esfuerzos (MPa)")
print(Stress)

0 1.5707963267948966
1 2.356194490192345
2 1.5707963267948966
3 0.0
4 0.7853981633974483
5 1.5707963267948966
Esfuerzos (MPa)
[-6.87549354  6.842404    4.78738069 -1.52788745 -2.88101221  4.78738069]


# Adding a non-stress bar

In [ ]:
NodesCondition = []
Nodes = 5
Nodes *= 2
NumberOfElement = 7

h = 1500 #mm
E = 3.2e5 #MPA
K = []
A = (0.25*np.pi*(50)**2)*np.ones(Nodes)
L = []
P_A = 5000
P_B = 4200
P_C = 2500
P_E = 3000

PosNodes = np.array([(0,0),(h,0),(0,h),(h,h),(h,2*h)])
Elements = np.array([(0,2),(1,2),(1,3),(2,3),(2,4),(3,4),(0,1)])

for i in range(0,NumberOfElement):
    L.append(DistNodes(PosNodes[Elements[i][0]],PosNodes[Elements[i][1]]))

L = np.array(L)


for i in range(0,NumberOfElement):
    aux = np.zeros((4,4))
    angle = L[i][1]
    rows = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    cols = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    for j in range(0,4):
        for k in range(0,4):
            aux[j][k] = rows[j]*cols[k]
    aux = aux*E*A[i]/L[i][0]
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(x=0,y=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en X
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+0) #Nodo 3 en X
UBoundaryCondition(U,0,2*1+1) #Nodo 3 en Y

FBoundaryCondition(F,-P_C,2*2+0) #Nodo 2 en X
FBoundaryCondition(F,-P_E,2*3+0) #Nodo 3 en X
FBoundaryCondition(F,-P_B,2*4+0) #Nodo 4 en X
FBoundaryCondition(F,P_A,2*4+1) #Nodo 4 en Y


U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)


Stiffness Matrix:
 [[ 4.18879020e+05  2.56489426e-11 -4.18879020e+05  0.00000000e+00
  -1.57054477e-27 -2.56489426e-11  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.56489426e-11  4.18879020e+05  0.00000000e+00  0.00000000e+00
  -2.56489426e-11 -4.18879020e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-4.18879020e+05  0.00000000e+00  5.66975118e+05 -1.48096098e+05
  -1.48096098e+05  1.48096098e+05 -1.57054477e-27 -2.56489426e-11
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.48096098e+05  5.66975118e+05
   1.48096098e+05 -1.48096098e+05 -2.56489426e-11 -4.18879020e+05
   0.00000000e+00  0.00000000e+00]
 [-1.57054477e-27 -2.56489426e-11 -1.48096098e+05  1.48096098e+05
   7.15071216e+05  2.91038305e-11 -4.18879020e+05  0.00000000e+00
  -1.48096098e+05 -1.48096098e+05]
 [-2.56489426e-11 -4.18879020e+05  1.48096098e+05 -1.48096098e+05
   2.91038305e-11  7.15071216e+05  0.00000000e+00  0.00000000e+00
  -1.48096098e

In [ ]:
ElementDisplacement = []
for i in range(0,NumberOfElement):
    Node1 = Elements[i][0]
    Node2 = Elements[i][1]
    ElementDisplacement.append([(U[2*Node1][0],U[2*Node1+1][0]),(U[2*Node2][0],U[2*Node2+1][0])])
Stress = []
for i in range(0,NumberOfElement):
    C = np.cos(L[i][1])
    S = np.sin(L[i][1])
    dd = ElementDisplacement[i]
    ss = -dd[0][0]*C - dd[0][1]*S + dd[1][0]*C+dd[1][1]*S
    Stress.append(ss*E/L[i][0])
Stress = np.array(Stress)
print("Esfuerzos (MPa)")
print(Stress)

Esfuerzos (MPa)
[-0.00707921  0.00698645  0.00468552 -0.00152789 -0.00302506  0.00468552
  0.        ]


El nuevo elemento (6) tiene esfuerzo 0, como se supone pues esta barra añadida no soporta ningún esfuerzo. Sin embargo, suele utilizarse por motivos de diseño frente a cargas excesivas.

# Changing conditions: Section F problem

In [ ]:
NodesCondition = []
Nodes = 5
Nodes *= 2
NumberOfElement = 6

h = 1200 #mm
E = 3.2e5 #MPA
K = []
A = (0.25*np.pi*(50)**2)*np.ones(Nodes)
L = []
P_A = 5400
P_B = 4000
P_C = 2500
P_E = 3000

PosNodes = np.array([(0,0),(h,0),(0,h),(h,h),(h,2*h)])
Elements = np.array([(0,2),(1,2),(1,3),(2,3),(2,4),(3,4)])

for i in range(0,NumberOfElement):
    L.append(DistNodes(PosNodes[Elements[i][0]],PosNodes[Elements[i][1]]))

L = np.array(L)


for i in range(0,NumberOfElement):
    aux = np.zeros((4,4))
    angle = L[i][1]
    rows = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    cols = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    for j in range(0,4):
        for k in range(0,4):
            aux[j][k] = rows[j]*cols[k]
    aux = aux*E*A[i]/L[i][0]
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(x=0,y=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en X
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+0) #Nodo 3 en X
UBoundaryCondition(U,0,2*1+1) #Nodo 3 en Y

FBoundaryCondition(F,-P_C,2*2+0) #Nodo 2 en X
FBoundaryCondition(F,-P_E,2*3+0) #Nodo 3 en X
FBoundaryCondition(F,-P_B,2*4+0) #Nodo 4 en X
FBoundaryCondition(F,P_A,2*4+1) #Nodo 4 en Y

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)


Stiffness Matrix:
 [[ 1.96318096e-27  3.20611782e-11  0.00000000e+00  0.00000000e+00
  -1.96318096e-27 -3.20611782e-11  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.20611782e-11  5.23598776e+05  0.00000000e+00  0.00000000e+00
  -3.20611782e-11 -5.23598776e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.85120122e+05 -1.85120122e+05
  -1.85120122e+05  1.85120122e+05 -1.96318096e-27 -3.20611782e-11
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.85120122e+05  7.08718898e+05
   1.85120122e+05 -1.85120122e+05 -3.20611782e-11 -5.23598776e+05
   0.00000000e+00  0.00000000e+00]
 [-1.96318096e-27 -3.20611782e-11 -1.85120122e+05  1.85120122e+05
   8.93839020e+05  2.91038305e-11 -5.23598776e+05  0.00000000e+00
  -1.85120122e+05 -1.85120122e+05]
 [-3.20611782e-11 -5.23598776e+05  1.85120122e+05 -1.85120122e+05
   2.91038305e-11  8.93839020e+05  0.00000000e+00  0.00000000e+00
  -1.85120122e

In [ ]:
ElementDisplacement = []
for i in range(0,NumberOfElement):
    Node1 = Elements[i][0]
    Node2 = Elements[i][1]
    ElementDisplacement.append([(U[2*Node1][0],U[2*Node1+1][0]),(U[2*Node2][0],U[2*Node2+1][0])])
Stress = []
for i in range(0,NumberOfElement):
    C = np.cos(L[i][1])
    S = np.sin(L[i][1])
    dd = ElementDisplacement[i]
    ss = -dd[0][0]*C - dd[0][1]*S + dd[1][0]*C+dd[1][1]*S
    Stress.append(ss*E/L[i][0])
Stress = np.array(Stress)
print("Esfuerzos (MPa)")
print(Stress)

Esfuerzos (MPa)
[-6.87549354  6.842404    4.78738069 -1.52788745 -2.88101221  4.78738069]
